In [2]:
import sys
sys.path.append("..") # Ensure the parent directory is in the path

import core.AssignmentTransformer as TransformerAnalysis
from core.DataLoader import DataPreprocessor
import core.AssignmentKFold as KFold
import numpy as np
from importlib import reload
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
MAX_JETS = 4
PLOTS_DIR = f"plots/"
import os
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

DataProcessor = DataPreprocessor(jet_features=["ordered_jet_pt", "ordered_jet_e", "ordered_jet_phi", "ordered_jet_eta", "ordered_jet_b_tag","m_l1j", "m_l2j", "dR_l1j", "dR_l2j"], 
                                lepton_features=["lep_pt","lep_e", "lep_eta", "lep_phi"],
                                jet_truth_label="ordered_event_jet_truth_idx", 
                                lepton_truth_label="event_lepton_truth_idx", 
                                global_features = ["met_met_NOSYS","met_phi_NOSYS"], 
                                max_leptons=2, 
                                max_jets = MAX_JETS, 
                                non_training_features= ["truth_ttbar_mass", "truth_ttbar_pt", "N_jets"], 
                                event_weight="weight_mc_NOSYS")
DataProcessor.load_data("/data/dust/group/atlas/ttreco/full_training.root", "reco", max_events=100000)
DataProcessor.prepare_data()
DataProcessor.normalise_data()
DataProcessor.split_data(0.2)

2025-07-11 14:36:37.762760: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-11 14:36:37.766549: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-11 14:36:37.777296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752237397.795084  282964 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752237397.800509  282964 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752237397.814564  282964 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
reload(TransformerAnalysis)
reload(KFold)

TransformerMatcher = TransformerAnalysis.TransformerJetMatcher(DataProcessor); TransformerMatcher.load_data(*DataProcessor.get_data())
TransformerMatcher.build_model(
    hidden_size=16,
    lep_dim=4,
    global_dim=2,
    jet_embedding_layers=2,
    lepton_embedding_layers=2,
    global_embedding_layers=2,
    attention_blocks=2,
    ff_layers=3,
    n_heads=6,
    dropout_rate=0,
    regularization_lambda=0.001,
)
TransformerMatcher.compile_model(
    lambda_excl=0, optimizer=keras.optimizers.Adam(learning_rate=0.001)
)
TransformerMatcher.model.summary()
TransformerMatcher.load_data(*DataProcessor.get_data())

In [ ]:
TransformerMatcher.train_model(epochs=100,
                                batch_size=128,
                                verbose=1,
                                weight = "sample",
                                callbacks = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=50, restore_best_weights=True, mode ="max"))


In [ ]:
TransformerMatcher.plot_history()

In [ ]:
TransformerMatcher.plot_confusion_matrix()

In [ ]:
TransformerMatcher.plot_permutation_importance(shuffle_number=5)